In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os; print(os.getcwd())
import socket; print(socket.gethostname())
from glob import glob

/home/ajasja/projects/truncator
dig60


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from glob import glob
import truncator



In [3]:
#import sys; sys.path.insert(0, "/home/ajasja/projects/Stable_Heterodimers")
#import utils as u

In [4]:
picks = glob("out/R3/30_fixed_loops/*.pdb")
for pick_name in picks:
    print('delete all')
    print(f"load '{pick_name}'")
    print('select loops, ss l')
    print('color red, loops')
    print("")

delete all
load 'out/R3/30_fixed_loops/ZCON_37__numH4.5__from-21.21__to22.24__ccA-DCB__llr2,5__ar1-2,0__ar2-3,0__rms0.5__ch-5__manloop.pdb'
select loops, ss l
color red, loops

delete all
load 'out/R3/30_fixed_loops/ZCON_127__numH4.5__from-20.77__to21.47__ccDA-BC__llr2,5__ar1-3,0__ar2-3,0__rms0.5__ch-5__manloop.pdb'
select loops, ss l
color red, loops

delete all
load 'out/R3/30_fixed_loops/ZCON_13__numH4.5__from-17.52__to22.91__ccBC-DA__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-5__manloop.pdb'
select loops, ss l
color red, loops

delete all
load 'out/R3/30_fixed_loops/ZCON_127__numH4.5__from-20.77__to21.47__ccC-DAB__llr2,4__ar1-2,0__ar2-3,0__rms0.5__ch-5__manloop.pdb'
select loops, ss l
color red, loops

delete all
load 'out/R3/30_fixed_loops/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__ccAD-CB__llr2,5__ar1-2,0__ar2-1,0__rms0.5__ch-5__manloop.pdb'
select loops, ss l
color red, loops

delete all
load 'out/R3/30_fixed_loops/ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar

In [5]:
def get_topology_from_name(name):
    name = truncator.basename_noext(name)
    #fifth field, skip first two cc chars
    gsplit=name.split('__')[4][2:]
    gsplit=gsplit.split('-')
    topology=str(len(gsplit[0]))+str(len(gsplit[1]))
    return topology
get_topology_from_name('out/R3/30_fixed_loops/ZCON_37__numH4.5__from-21.21__to22.24__ccAB-CD__llr2,5__ar1-3,0__ar2-2,0__rms0.5__ch-5__manloop.pdb')=='22'

def get_charges_from_name(name):
    topology = get_topology_from_name(name)
    if topology=='22':
        return 'truncator/xml/-5.charge','truncator/xml/-5.charge'
    if topology=='13':
        return 'truncator/xml/-3.charge','truncator/xml/-7.charge'
    if topology=='31':
        return 'truncator/xml/-7.charge','truncator/xml/-3.charge'
    return None
get_charges_from_name('out/R3/30_fixed_loops/ZCON_37__numH4.5__from-21.21__to22.24__ccAB-CD__llr2,5__ar1-3,0__ar2-2,0__rms0.5__ch-5__manloop.pdb')

('truncator/xml/-5.charge', 'truncator/xml/-5.charge')

In [7]:
%%time
struct_name = picks[5]

chargeA, chargeB = get_charges_from_name(struct_name)
result=truncator.fix_surface(struct_name, out_dir='out/R3/test_charges', script_name='truncator/xml/31_fix_surf.xml', 
                                                  rosetta_bin="/software/rosetta/latest/bin/rosetta_scripts",
                                                  charge_constraints_chA=chargeA,
                                                  charge_constraints_chB=chargeB
                            );

print(result.score_file)


cd out/R3/test_charges/ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-5__ch-5; /software/rosetta/latest/bin/rosetta_scripts  \
-parser:protocol /home/ajasja/projects/truncator/truncator/xml/31_fix_surf.xml \
-s /home/ajasja/projects/truncator/out/R3/30_fixed_loops/ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-5.pdb \
-out:suffix __ch-5 \
-out:file:scorefile  ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-5__ch-5.sc \
-beta \
-in:file:fullatom \
-renumber_pdb 1 \
-out:file:pdb_comments true \
-run:preserve_header true \
-out:file:scorefile_format json \
-out:pdb \
-out:no_nstruct_label \
-parser:script_vars charge_constraints_chA="/home/ajasja/projects/truncator/truncator/xml/-3.charge" charge_constraints_chB="/home/ajasja/projects/truncator/truncator/xml/-7.charge"  \
-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation VirtualBB ShoveBB VirtualDNAPhosphate Virtual

In [6]:
import dask, distributed
client = distributed.Client(scheduler_file='scheduler.json'); client

Client Scheduler: tcp://172.16.131.90:13443 Dashboard: http://172.16.131.90:8787/status,Cluster Workers: 8 Cores: 8 Memory: 16.00 GB


In [7]:
res = []
for struct_name in picks:
    chargeA, chargeB = get_charges_from_name(struct_name)
    res.append(client.submit(
        truncator.fix_surface,struct_name, out_dir='out/R3/34_fix_surf_fixed_loops_charges_50runs', script_name='truncator/xml/31_fix_surf.xml', 
                                                  rosetta_bin="/software/rosetta/latest/bin/rosetta_scripts",
                                                  charge_constraints_chA=chargeA,
                                                  charge_constraints_chB=chargeB, pure=False)
                 );

distributed.utils - ERROR - can't start new thread
Traceback (most recent call last):
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/distributed/utils.py", line 648, in log_errors
    yield
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/distributed/client.py", line 873, in _reconnect
    yield self._ensure_connected()
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/distributed/client.py", line 888, in _ensure_connected
    connection_args=self.connection_args)
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/ajasja/.conda/envs

In [30]:
try:
    distributed.wait(res, timeout=1);
    print("FINISHED!")
except:
    print("Not Yet Finished")

Not Yet Finished


In [17]:
try:
    distributed.wait(res, timeout=1);
except:
    print("Not Yet Finished")

Not Yet Finished


In [8]:
#print("\n".join(list(df.columns)));
cols = truncator.read_file_lines('col_order.list', skip_comments=True)
df = df[cols]


In [9]:
len(df)

6129